In [1]:
from pathlib import Path

import pandas as pd

# Paths
PROJECT_ROOT = Path("..").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_RAW, DATA_PROCESSED


(PosixPath('/Users/arnavjain/manutd-2007-08-style-profile/data/raw'),
 PosixPath('/Users/arnavjain/manutd-2007-08-style-profile/data/processed'))

In [3]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
DATA_RAW = PROJECT_ROOT / "data" / "raw"

standard = pd.read_csv(DATA_RAW / "manutd_2007_08_standard_stats_sample.csv")
shooting = pd.read_csv(DATA_RAW / "manutd_2007_08_shooting.csv")
goalkeeping = pd.read_csv(DATA_RAW / "manutd_2007_08_goalkeeping.csv")
playing_time = pd.read_csv(DATA_RAW / "manutd_2007_08_playing_time.csv")
misc = pd.read_csv(DATA_RAW / "manutd_2007_08_misc.csv")
matches = pd.read_csv(DATA_RAW / "manutd_2007_08_matches_raw.csv")

standard.head(), shooting.head(), goalkeeping.head()


(  Playing Time Performance Per 90 Minutes Unnamed: 3
 0          G+A        G-PK         G+A-PK    Matches
 1         0.09        0.06           0.09    Matches
 2         0.09        0.03           0.09    Matches
 3         0.03        0.00           0.03    Matches
 4         0.07        0.03           0.07    Matches,
               Player   Nation    Pos  Age   90s  Gls  Sh  SoT  SoT%  Sh/90  \
 0      Rio Ferdinand  eng ENG     DF   28  33.8    2 NaN    6   NaN   0.18   
 1          Wes Brown  eng ENG     DF   27  33.7    1 NaN    4   NaN   0.12   
 2       Patrice Evra   fr FRA     DF   26  31.3    0 NaN    2   NaN   0.06   
 3      Nemanja Vidić   rs SRB     DF   25  29.6    1 NaN    5   NaN   0.17   
 4  Cristiano Ronaldo   pt POR  FW,MF   22  30.5   31 NaN   89   NaN   2.92   
 
    SoT/90  G/Sh  G/SoT  Dist  PK    PKatt  Matches  
 0     NaN  0.33    NaN     0   0  Matches      NaN  
 1     NaN  0.25    NaN     0   0  Matches      NaN  
 2     NaN  0.00    NaN     0   0  Ma

In [4]:
import pandas as pd

print("STANDARD:", pd.read_csv(DATA_RAW/"manutd_2007_08_standard_stats_sample.csv").columns.tolist())
print("SHOOTING:", pd.read_csv(DATA_RAW/"manutd_2007_08_shooting.csv").columns.tolist())
print("GOALKEEPING:", pd.read_csv(DATA_RAW/"manutd_2007_08_goalkeeping.csv").columns.tolist())
print("PLAYING TIME:", pd.read_csv(DATA_RAW/"manutd_2007_08_playing_time.csv").columns.tolist())
print("MISC:", pd.read_csv(DATA_RAW/"manutd_2007_08_misc.csv").columns.tolist())


STANDARD: ['Playing Time', 'Performance', 'Per 90 Minutes', 'Unnamed: 3']
SHOOTING: ['Player', 'Nation', 'Pos', 'Age', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'PK', 'PKatt', 'Matches']
GOALKEEPING: ['Player', 'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'SoTA', 'Saves', 'Save%', 'W', 'D', 'L', 'CS', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Save%.1', 'Matches']
PLAYING TIME: ['Player', 'Nation', 'Pos', 'Age', 'MP', 'Min', 'Mn/MP', 'Min%', '90s', 'Starts', 'Mn/Start', 'Compl', 'Subs', 'Mn/Sub', 'unSub', 'PPM', 'onG', 'onGA', '+/-', '+/-90', 'On-Off', 'Matches']
MISC: ['Player', 'Nation', 'Pos', 'Age', '90s', 'CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW', 'PKwon', 'PKcon', 'OG', 'Matches']
